In [1]:
import requests
from PIL import Image
import base64
import requests
import os
from dotenv import load_dotenv
from main_function import generate_image_from_text, resize_image, generate_and_download_video

## Image generation

In [6]:
# Load environment variables from .env file
#load_dotenv()
#api_key = os.getenv("API_KEY")

In [2]:
api_key='sk-sCTLqlnzDxEQJrHmtT3GTNncfLYX5n9njuL04oWKbav0PUqR'

In [3]:
# Combined usage:
text_prompts = [
    {"text": "Ten years have passed. Harry, now almost eleven, has been living in a small cupboard under the stairs. Harry has been abused by the Mr. and Mrs. Dursley, as well as their only son, Dudley. Dudley is spoiled and whiny. Today is Dudley’s birthday, and he complains about getting only 37 presents this year. Harry is very self-conscience, and the only thing he likes about his appearance is a lightning shaped scar on his forehead. He is told he got that in a car crash when his parents died.", "weight": 1},
    {"text": "blurry, bad", "weight": -1},]
output_directory = "Images/"
book_name = "Harry"

In [4]:
#generate_image_from_text(api_key, text_prompts, output_directory, book_name)

In [5]:
#resize_image("Images/Harry_2831275650.png")

In [4]:
generate_and_download_video(api_key, text_prompts, "Images", "1")

Exception: Non-200 response: {"id":"f66b4d68c65ef97869971859cf44c1ad","message":"for stable-diffusion-xl-1024-v0-9 and stable-diffusion-xl-1024-v1-0 the allowed dimensions are 1024x1024, 1152x896, 1216x832, 1344x768, 1536x640, 640x1536, 768x1344, 832x1216, 896x1152, but we received 768x768","name":"invalid_sdxl_v1_dimensions"}


++++++++++++++++++++++++++++++

In [13]:
#api_key = "sk-bkuIqt0pQzG17G7H3q8phhkvYLBkUK3S7z4vqMfLtCdEjkDJ"
response = requests.post(
    f"https://api.stability.ai/v2alpha/generation/image-to-video",
    headers={"authorization": f"Bearer {api_key} "},
    #files={"image": open("./animal_farm.jpeg", "rb")},
    files={"image": open("Images/Harry_2831275650.png", "rb")},
    data={
        "seed": 0,
        "cfg_scale": 1.8,
        "motion_bucket_id": 127
    },
)

generation_id = response.json().get('id')
print("Generation ID:", response.json().get('id'))

Generation ID: 1d61aa10bad4ebdc820fe1aecac5a24068661aaf3e894d394e3bf8b693625609


In [ ]:
response = requests.request(
    "GET",
    f"https://api.stability.ai/v2alpha/generation/image-to-video/result/{generation_id}",
    headers={
        'Accept': "video/*",  # Use 'application/json' to receive base64 encoded JSON
        'authorization': f"Bearer {api_key}"
    },
)

if response.status_code == 202:
    print("Generation in-progress, try again in 10 seconds.")
elif response.status_code == 200:
    print("Generation complete!")
    with open("video.mp4", 'wb') as file:
        file.write(response.content)
else:
    raise Exception(str(response.json()))

In [ ]:
def generate_video(api_key, image_path):
    response = requests.post(
        f"https://api.stability.ai/v2alpha/generation/image-to-video",
        headers={"authorization": f"Bearer {api_key} "},
        files={"image": open("./animal_farm.jpeg", "rb")},
        data={
            "seed": 0,
            "cfg_scale": 1.8,
            "motion_bucket_id": 127
        },
    )
    
generation_id = response.json().get('id')
print("Generation ID:", response.json().get('id'))

Combined function